# Python events on a map
Events from: https://wiki.python.org/moin/PythonEventsCalendar section "iCal Downloads"

To run the whole notebook you need:
`pip install icalendar mapboxgl funcy`
and clone or download zip of geojson polygons for countries from here
https://github.com/LonnyGomes/CountryGeoJSONCollection/tree/master/geojson

## Show me the map!
Ok, to see a map, you can use already generated file
You need to do two things:
* get a mapbox token
* Fast-forward to last 2 cells and run them

In [104]:
import icalendar

In [105]:
with open('./user groups.ics', 'rb') as icalfile:
    ug = icalendar.Calendar.from_ical(icalfile.read())

ug_events = {}

for component in ug.walk():
    summary = component.get('summary')
    ug_events[summary] = {
        'description': component.get('description'),
        'location': component.get('location'),
        'startdt': component.get('dtstart'),        
    }



In [106]:
with open('./main events.ics', 'rb') as icalfile:
    me = icalendar.Calendar.from_ical(icalfile.read())

main_events = {}

for component in me.walk():
    summary = component.get('summary')
    main_events[summary] = {
        'description': component.get('description'),
        'location': component.get('location'),
        'startdt': component.get('dtstart'),        
    }



In [107]:
import os
from pathlib import Path
import json

from funcy import project

## Path to polygons
Change the path to polygons in next block to run locally, they're too big to include in github dist.

In [117]:
countries_path = Path('./CountryGeoJSONCollection-master/geojson/')

In [108]:
countries = {}
for file in countries_path.iterdir():
    with open(file) as geojson_file:
        try:
            geojson_data = json.load(geojson_file)
            country_name = geojson_data['properties']["SUBUNIT"].lower()
            countries[country_name] = geojson_data
            countries[country_name]['properties']['main_events'] = 0
            countries[country_name]['properties']['group_events'] = 0
        except:
            pass

In [112]:
for event in main_events.values():
    event_country = str(event['location']).split(',')[-1].strip().lower()
    if event_country in countries:
        countries[event_country]['properties']['main_events'] = countries[event_country]['properties'].get('main_events', 0) + 1
    if event_country in ['us', 'usa', 'united states']:
        countries['united states of america']['properties']['main_events'] = countries['united states of america']['properties'].get('main_events', 0) + 1
    

In [113]:
for event in ug_events.values():
    event_country = str(event['location']).split(',')[-1].strip().lower()
    if event_country in countries:
        countries[event_country]['properties']['group_events'] = countries[event_country]['properties'].get('group_events', 0) + 1
    if event_country in ['us', 'usa', 'united states']:
        countries['united states of america']['properties']['group_events'] = countries['united states of america']['properties'].get('group_events', 0) + 1
    

In [114]:
countries_geo_collection = {
    'type': 'FeatureCollection',
    'features': [
        {
            'geometry': country['geometry'],
            'properties': project(country['properties'], ['NAME_LONG', 'main_events', 'group_events'])
        }
        for country in countries.values()
        if country['properties']['main_events'] or country['properties']['group_events']
    ],
}

with open('python_events_by_country.geojson', 'w') as outfile:
    json.dump(countries_geo_collection, outfile, indent=2)


## Maps comes next
To move in 3D space: hold control, click the mouse and move

To run locally you need to obtain access token from mapbox and have it in env as `MAPBOX_ACCESS_TOKEN`
To obtain a token, create an account here: http://mapbox.com/

In [115]:
import os
from mapboxgl.viz import *
from mapboxgl.utils import *

# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

In [118]:
viz = ChoroplethViz(
    'python_events_by_country.geojson', 
    color_property='group_events',
    color_stops=create_color_stops([0, 1, 10, 20], colors='YlOrRd'),
    color_function_type='interpolate',
    line_stroke='--',
    color_default='521253',
    line_color='rgb(128,0,38)',
    line_width=1,
    opacity=0.95,
    center=(0,40),
    zoom=0.8,
    below_layer='waterway-label',
    height_property='main_events',
    height_stops=[[0, 0], [1, 10000], [3, 75000], [5, 100000], [10, 250000], [20, 500000]],
)
viz.show()